In [18]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import logging
import requests
import json
import pandas as pd

# Load environment variables
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)

In [19]:
def setup_genai():
    google_api_key = os.getenv('GOOGLE_API_KEY')
    if not google_api_key:
        logging.error("Google API key is not set.")
        raise ValueError("Google API key is not set.")
    genai.configure(api_key=google_api_key)
    return genai

# def download_file(url, destination):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Raises an HTTPError for bad responses
#         with open(destination, 'wb') as f:
#             f.write(response.content)
#         logging.info(f"File downloaded successfully: {destination}")
#     except requests.RequestException as e:
#         logging.error(f"Failed to download file: {e}")
#         raise

def upload_audio_ads(audio_ad_paths):
    genai_client = setup_genai()
    uploaded_files = []
    for path in audio_ad_paths:
        try:
            file_handle = genai_client.upload_file(path=path)
            uploaded_files.append(file_handle)
            logging.info(f"Audio ad uploaded successfully: {path}")
        except Exception as e:
            logging.error(f"Error uploading audio ad: {e}")
    return uploaded_files

def generate_ad_script(uploaded_files, context):
    genai_client = setup_genai()
    try:
        model = genai_client.GenerativeModel('models/gemini-1.5-pro-latest')
        prompt = f"Generate an engaging audio ad script based on the provided context: '{context}'. Use the uploaded audio ads as references to guide the script generation."
        response = model.generate_content([prompt] + uploaded_files)
        return response.text
    except Exception as e:
        logging.error(f"Error generating ad script: {e}")
        raise

def save_ad_script(ad_script, output_path):
    try:
        with open(output_path, 'w') as f:
            f.write(ad_script)
        logging.info(f"Ad script saved successfully: {output_path}")
    except Exception as e:
        logging.error(f"Error saving ad script: {e}")
        raise

def get_user_input():
    company_name = input("What is your company name and what does it do? ")
    product_details = input("What is the product you are selling and what does it do? ")
    organization_size = input("How large is your organization? ")
    product_differentiator = input("What is one differentiator for your product compared to the marketplace? ")
    
    return {
        "company_name": company_name,
        "product_details": product_details,
        "organization_size": organization_size,
        "product_differentiator": product_differentiator
    }


In [20]:
if __name__ == "__main__":
    user_responses = get_user_input()
    audio_ad_paths = ["Apple_Ad_1.mp3", "Apple_Ad_2.mp3", "Radio_Shack_Ad.mp3"]
    
    uploaded_files = upload_audio_ads(audio_ad_paths)
    
    # Building a more detailed context string
    context = (
        f"Introducing a new product from {user_responses['company_name']}, designed for {user_responses['product_details']}. "
        f"Our company size is {user_responses['organization_size']} and we stand out by {user_responses['product_differentiator']}."
    )
    
    ad_script = generate_ad_script(uploaded_files, context)
    print("Generated Ad Script:")
    print(ad_script)
    
    output_path = "generated_ad_script.txt"
    save_ad_script(ad_script, output_path)


INFO:root:Audio ad uploaded successfully: Apple_Ad_1.mp3
INFO:root:Audio ad uploaded successfully: Apple_Ad_2.mp3
INFO:root:Audio ad uploaded successfully: Radio_Shack_Ad.mp3
INFO:root:Ad script saved successfully: generated_ad_script.txt


Generated Ad Script:
(Upbeat music fades in, then plays under throughout)

**Narrator:** Are you tired of the same old boring furniture? Does your living room lack a touch of modern elegance? 

**Then it's time to experience the future of furniture with the Sofa 3000 from Felix, Inc.!**

**(Music swells slightly)**

**Narrator:**  Felix, Inc., a leader in innovative design with 6,000 employees dedicated to crafting exceptional pieces, proudly presents the Sofa 3000.  This isn't your grandma's couch. This is a statement piece.  Crafted entirely from sleek, lightweight aluminum, the Sofa 3000 offers a unique combination of style and durability.

**But that's not all!**

**(Music becomes slightly more playful)**

**Narrator:** The Sofa 3000 is also modular, meaning you can customize it to fit your exact space and style.  Need a cozy loveseat for date night? No problem.  Expecting a crowd for the big game? Easy! Just add a few extra modules and you're ready to go. 

**The Sofa 3000 is more